In [18]:
import subprocess

def generate_ast(source_code_file, output_file):
    subprocess.run(['srcml', source_code_file, '-o', output_file])

source_code_file = 'test1.cpp'
output_file = 'ast1.xml'

# Generate AST
generate_ast(source_code_file, output_file)

In [2]:
import xml.etree.ElementTree as ET

# Define the namespace globally
NAMESPACE = {'src': 'http://www.srcML.org/srcML/src'}

def parse_xml(xml_file):
    """Parse the XML file and return the root element."""
    tree = ET.parse(xml_file)
    return tree.getroot()

def count_function_declarations(root):
    """Count the number of function declarations in the XML."""
    count = 0
    for function_name in root.findall('.//src:function/src:name', namespaces=NAMESPACE):
        if function_name.text != "main":
            count += 1
    return count

def get_parameter_count_in_functions(root):
    """Get the total parameter count across all functions."""
    total_parameter_count = 0
    for function in root.findall('.//src:function', namespaces=NAMESPACE):
        parameter_list = function.find("./src:parameter_list", namespaces=NAMESPACE)
        # Count the number of parameters in the parameter list for each function
        parameter_count = sum(1 for _ in parameter_list.findall('./src:parameter', namespaces=NAMESPACE))
        # Add the parameter count for the current function to the total count
        total_parameter_count += parameter_count
    return total_parameter_count


def count_simple_pointers(root):
    """Count the number of simple pointers."""
    count = 0
    parameters = root.findall('.//src:parameter', namespaces=NAMESPACE)
    for parameter in parameters:
        modifier = parameter.find('.//src:modifier', namespaces=NAMESPACE)
        if modifier is not None and modifier.text == '*':
            count += 1
    return count

def count_simple_addresses(root):
    """Count the number of simple addresses."""
    count = 0
    function_calls = root.findall('.//src:call', namespaces=NAMESPACE)
    for call in function_calls:
        arguments = call.findall('.//src:argument', namespaces=NAMESPACE)
        for argument in arguments:
            address_operator = argument.find('.//src:operator', namespaces=NAMESPACE)
            if address_operator is not None:
                count += 1
    return count

def count_array_pointers(root):
    """Count the number of array pointers."""
    count = 0
    parameters = root.findall('.//src:parameter', namespaces=NAMESPACE)
    for parameter in parameters:
        parameter_name = parameter.find('.//src:decl/src:name', namespaces=NAMESPACE)
        if parameter_name is not None and parameter_name.find('src:index', namespaces=NAMESPACE) is not None:
            count += 1
    return count


In [3]:
import json

def get_quality_metrics(xml_file):
    # Parse the XML file
    root = parse_xml(xml_file)

    # Call each function to get the feature values
    # nf = count_function_declarations(root)
    # np = get_parameter_count_in_functions(root)
    # npt_s = count_simple_pointers(root)
    # nadd_s = count_simple_addresses(root)
    # npt_ds = count_array_pointers(root)

    # Construct the JSON object
    quality_metrics = {
        'NF': count_function_declarations(root),
        'NP': get_parameter_count_in_functions(root),
        'NPt-s': count_simple_pointers(root),
        'NAdd-s': count_simple_addresses(root),
        'NPt-ds': count_array_pointers(root)
    }
    print(quality_metrics)
    # return json.dumps(quality_metrics, indent=4)
    quality_metrics_filtered = {k: v for k, v in quality_metrics.items() if v >= 1}

    return quality_metrics_filtered

{'NF': 3, 'NP': 3, 'NPt-s': 0, 'NAdd-s': 1, 'NPt-ds': 0}
Quality metrics saved to quality_metrics.json


In [ ]:
# Example usage
xml_file = "ast.xml"  # Provide the path to your srcML XML file
json_metrics = get_quality_metrics(xml_file)
output_file = "quality_metrics.json"
with open(output_file, "w") as json_file:
    json.dump(json_metrics, json_file, indent=4)

print(f"Quality metrics saved to {output_file}")

In [5]:
FUNCTION_MAP = {
    'NF': count_function_declarations,
    'NP': get_parameter_count_in_functions,
    'NPt-s': count_simple_pointers,
    'NAdd-s': count_simple_addresses,
    'NPt-ds': count_array_pointers
}

In [10]:
def extract_features(root,json_file):
    with open(json_file, 'r') as f:
        data = json.load(f)
    
    features = {}
    for key, func in FUNCTION_MAP.items():
        if key in data:
            features[key] = func(root)
    
    return features

In [19]:
json_file = "quality_metrics.json"  # Path to the generated JSON file
xml_file = "ast1.xml"
root =  parse_xml(xml_file)
extracted_features = extract_features(root, json_file)

print("Extracted features:")
print(extracted_features)

Extracted features:
{'NF': 3, 'NP': 3, 'NAdd-s': 0}


In [7]:
def calculate_individual_scores(teacher_solution, student_solution):
    individual_scores = {}
    
    for feature, teacher_count in teacher_solution.items():
        if feature in student_solution:
            student_count = student_solution[feature]
            # Calculate the score based on the difference between counts
            feature_score = max(0, min(1, 1 - abs(teacher_count - student_count) / teacher_count))
            # Store the individual score for the feature
            individual_scores[feature] = f"{student_count}/{teacher_count}"
        else:
            individual_scores[feature] = f"0/{teacher_count}"
    
    return individual_scores

# Example usage:
teacher_solution = {'NF': 5, 'NP': 10, 'NPt-s': 3, 'NAdd-s': 2, 'NPt-ds': 1}
student_solution = {'NF': 4, 'NP': 9, 'NPt-s': 3, 'NAdd-s': 1, 'NPt-ds': 1}

individual_scores = calculate_individual_scores(teacher_solution, student_solution)
print("Individual scores:", individual_scores)


Individual scores: {'NF': '4/5', 'NP': '9/10', 'NPt-s': '3/3', 'NAdd-s': '1/2', 'NPt-ds': '1/1'}


In [9]:
def calculate_individual_scores(teacher_solution, student_solution):
    individual_scores = {}
    total_score = 0
    
    for feature, teacher_count in teacher_solution.items():
        if feature in student_solution:
            student_count = student_solution[feature]
            # Calculate the score based on the difference between counts
            feature_score = max(0, min(1, 1 - abs(teacher_count - student_count) / teacher_count))
            # Add the feature score to the total score
            total_score += feature_score
            # Store the individual score for the feature
            individual_scores[feature] = {
                "student_count": student_count,
                "teacher_count": teacher_count,
                "score" : f"{student_count}/{teacher_count}",
                "feature_score": feature_score
            }
        else:
            # If the feature is not present in the student's solution
            individual_scores[feature] = {
                "student_count": 0,
                "teacher_count": teacher_count,
                "feature_score": 0
            }
    
    # Calculate the final score percentage
    num_features = len(teacher_solution)
    final_score_percentage = total_score / num_features * 100 if num_features > 0 else 0
    
    return individual_scores, final_score_percentage

# Example usage:
teacher_solution = {'NF': 5, 'NP': 10, 'NPt-s': 3, 'NAdd-s': 2, 'NPt-ds': 1}
student_solution = {'NF': 4, 'NP': 9, 'NPt-s': 3, 'NAdd-s': 1, 'NPt-ds': 1}

individual_scores, final_score_percentage = calculate_individual_scores(teacher_solution, student_solution)
print("Individual scores:", individual_scores)
print("Final score percentage:", final_score_percentage)


Individual scores: {'NF': {'student_count': 4, 'teacher_count': 5, 'score': '4/5', 'feature_score': 0.8}, 'NP': {'student_count': 9, 'teacher_count': 10, 'score': '9/10', 'feature_score': 0.9}, 'NPt-s': {'student_count': 3, 'teacher_count': 3, 'score': '3/3', 'feature_score': 1}, 'NAdd-s': {'student_count': 1, 'teacher_count': 2, 'score': '1/2', 'feature_score': 0.5}, 'NPt-ds': {'student_count': 1, 'teacher_count': 1, 'score': '1/1', 'feature_score': 1}}
Final score percentage: 84.00000000000001


In [11]:
def calculate_individual_scores(teacher_solution, student_solution):
    individual_scores = {}
    total_score = 0
    
    for feature, teacher_count in teacher_solution.items():
        if feature in student_solution:
            student_count = student_solution[feature]
            # Calculate the score based on the difference between counts
            feature_score = max(0, min(1, 1 - abs(teacher_count - student_count) / teacher_count))
            # Penalize if the student's count exceeds the teacher's count
            if student_count > teacher_count:
                feature_score *= 0.5  # Example penalty factor (adjust as needed)
            # Add the feature score to the total score
            total_score += feature_score
            # Store the individual score for the feature
            individual_scores[feature] = {
                "student_count": student_count,
                "teacher_count": teacher_count,
                "score": f"{student_count}/{teacher_count}",
                "feature_score": feature_score
            }
        else:
            # If the feature is not present in the student's solution
            individual_scores[feature] = {
                "student_count": 0,
                "teacher_count": teacher_count,
                "score": f"0/{teacher_count}",
                "feature_score": 0
            }
    
    # Calculate the final score percentage
    num_features = len(teacher_solution)
    final_score_percentage = total_score / num_features * 100 if num_features > 0 else 0
    print(total_score,num_features)
    return individual_scores, final_score_percentage

# Example usage:
teacher_solution = {'NF': 5, 'NP': 10, 'NPt-s': 3, 'NAdd-s': 2, 'NPt-ds': 1}
student_solution = {'NF': 4, 'NP': 12, 'NPt-s': 2, 'NAdd-s': 3, 'NPt-ds': 1}

individual_scores, final_score_percentage = calculate_individual_scores(teacher_solution, student_solution)
print("Individual scores:", individual_scores)
print("Final score percentage:", final_score_percentage)


3.116666666666667 5
Individual scores: {'NF': {'student_count': 4, 'teacher_count': 5, 'score': '4/5', 'feature_score': 0.8}, 'NP': {'student_count': 12, 'teacher_count': 10, 'score': '12/10', 'feature_score': 0.4}, 'NPt-s': {'student_count': 2, 'teacher_count': 3, 'score': '2/3', 'feature_score': 0.6666666666666667}, 'NAdd-s': {'student_count': 3, 'teacher_count': 2, 'score': '3/2', 'feature_score': 0.25}, 'NPt-ds': {'student_count': 1, 'teacher_count': 1, 'score': '1/1', 'feature_score': 1}}
Final score percentage: 62.33333333333334
